In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

response = client.chat.completions.create(
  model="deepseek-r1:1.5b",
  messages=[
    {"role": "system", "content": " You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The LA Dodgers won in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)
print(response.choices[0].message.content)

<think>

</think>

The movie "It" (2014) isDirected by360°, starred in its original form with Vlad interchange with a mix of different countries. The original theatrical version is called *Famusz* and its official title is *It: Maternal Labor and the Rise of Modern Family Dynamics*. 

A film adaptation has been released as *Family* (2018), which features some original cast members, but it's also available on other platforms like Amazon Prime.


In [16]:
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_core.output_parsers import StrOutputParser

# Step 1: Initialize the Ollama model
chat_model = ChatOllama(model="llama3.2:latest")  # Replace with your Ollama model

# Step 2: Define a prompt for query expansion
promp_template = ChatPromptTemplate([
                (
                         "system",
                         "you are a kind of 5 years with a deeper understanding on physics and mathematics. And you explein the concept any in a simple way with similitudes for kids."
                ),
                (
                        "user",
                        "List[str]: List of paraphrased variations of the question : {question}" 

                )
                ])
# Step 3: Create a chain for query expansion
expansion_chain = promp_template | chat_model| StrOutputParser()

# Step 4: Define a function to refine the expanded query
def refine_expansion(output: str) -> str:
    # Remove unnecessary text and clean up the output
    expanded_terms = output.strip().split("\n")
    expanded_terms =[term.strip() for term in expanded_terms if term.strip()]
    #eval(output.strip().split("# Output:")[1])
   # [term.strip() for term in expanded_terms if term.strip()]
    return expanded_terms



In [19]:
# Step 5: Create the full pipeline
def query_expansion_pipeline(query: str) -> str:
    # Generate expanded terms
    expansion_output = expansion_chain.invoke(query)
    # Refine the output
    #refined_expansion = refine_expansion(expansion_output)
    return  expansion_output



In [20]:
# Step 6: Test the pipeline
query = "what is rag"
expanded_query = query_expansion_pipeline(query)
reponse=expanded_query
print(f"Original Query: {query}")
print(f"Expanded Query: {reponse}")

Original Query: what is rag
Expanded Query: Here are some paraphrased variations of the question "What is Rag?" that I came up with:

1. **What's a Rag?** (simple and direct)
2. **Can you tell me about Rag?** (inquisitive tone, for kids to ask their parents or teacher)
3. **I don't understand what Rag means... Can you explain?** (to show that I know there's more to it than just the word itself)
4. **What's the difference between a Rag and something else?** (to encourage critical thinking and comparison skills)
5. **Can you give me examples of things called Rag?** (to help kids visualize and connect with the concept)

Now, about what Rag actually is... 

**Rag**: A rag is an old piece of cloth used for wiping, cleaning, or as a material for crafts. Think of it like a soft, absorbent sponge! We use rags in our daily lives to clean up spills, wipe surfaces, or even make art projects.

But wait, there's more!

**Physics Connection**: You know how sometimes we stretch or compress materials,

In [24]:
(reponse).split("\n")

['Here are some paraphrased variations of the question "What is Rag?" that I came up with:',
 '',
 "1. **What's a Rag?** (simple and direct)",
 '2. **Can you tell me about Rag?** (inquisitive tone, for kids to ask their parents or teacher)',
 "3. **I don't understand what Rag means... Can you explain?** (to show that I know there's more to it than just the word itself)",
 "4. **What's the difference between a Rag and something else?** (to encourage critical thinking and comparison skills)",
 '5. **Can you give me examples of things called Rag?** (to help kids visualize and connect with the concept)',
 '',
 'Now, about what Rag actually is... ',
 '',
 '**Rag**: A rag is an old piece of cloth used for wiping, cleaning, or as a material for crafts. Think of it like a soft, absorbent sponge! We use rags in our daily lives to clean up spills, wipe surfaces, or even make art projects.',
 '',
 "But wait, there's more!",
 '',
 "**Physics Connection**: You know how sometimes we stretch or compr

In [ ]:
variations = [result for result in expanded_query]

In [30]:
from langchain_core.globals import set_llm_cache
from langchain_core.caches import InMemoryCache
# Set up cache
set_llm_cache(InMemoryCache())
query_cache: Dict[str, Tuple[str, float]] = {}


NameError: name 'Dict' is not defined

In [6]:
expanded_query

'<think>\nAlright, so I need to expand the term "rag" for better search results. Let me think about all possible meanings and related terms that might be associated with "rag." \n\nFirst off, "rag" can refer to animal hair. That\'s straightforward, but maybe adding something about its texture could help. Words like curly, fine, silky come to mind because they describe the appearance of animal hair.\n\nI also remember that in art or painting, "rag" might be used metaphorically to describe a rough, unpolished surface. So expanding on that with terms like weathered, worn away sounds useful.\n\nIn gaming contexts, particularly card games, "rag" can mean a specific card. That\'s another relevant term, so including it makes sense. It\'s probably used when discussing playing cards or strategy games.\n\nI should also consider the physical aspects—like how tightly wound or thick it is could be important terms for someone searching for information on ragging. So adding terms like tight wound and

In [10]:
def filter_terms(terms: list, context: str) -> list:
    # Example: Filter terms based on relevance to the context
    relevant_terms = [term for term in terms if context.lower() in term.lower()]
    return relevant_terms
def query_expansion_pipeline(query: str, context: str) -> str:
    # Generate expanded terms
    expansion_output = expansion_chain.run(query=query, context=context)
    # Refine the output
    expanded_terms = refine_expansion(expansion_output).split(", ")
    # Filter terms based on context
    filtered_terms = filter_terms(expanded_terms, context)
    return ", ".join(filtered_terms)

# Test the advanced pipeline
query = "machine learning"
context = "deep learning"
expanded_query = query_expansion_pipeline(query, context)
print(f"Original Query: {query}")
print(f"Expanded Query (Filtered): {expanded_query}")

Original Query: machine learning
Expanded Query (Filtered): like clustering algorithms.

Deep learning is another area within machine learning. It uses neural networks with many layers. That sounds a bit more complex than just regular learning. So maybe that should be on the list too.

I also think about practical applications of ML. Classification comes to mind—like predicting whether something will happen or not, deep learning, that should cover the expanded terms without missing anything.
</think>

machine learning expanded terms or phrases:

- Supervised Learning
- Unsupervised Learning
- Deep Learning
- Classification
- Clustering
- Reinforcement Learning
- Natural Language Processing
- Ethical Considerations


In [ ]:
variations = [result.paraphrased_query for result in results]